In [1]:
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import os

In [2]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [3]:
images_sad = os.listdir('../face_sad')
images_happy = os.listdir('../face_happy')
images_neutral = os.listdir('../face_neutral')
images_angry = os.listdir('../face_angry')
images_surprised = os.listdir('../face_surprised')
images_disgusted = os.listdir('../face_disgusted')

print(len(images_sad), len(images_happy), len(images_neutral), len(images_angry), len(images_surprised), len(images_disgusted))

8391 8996 10004 7980 7999 7999


In [4]:
def inference_facemesh(image):
    try:
        frame = image.copy()
    except AttributeError:
        return None
    H, W, _ = frame.shape
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_mesh = face_mesh.process(rgb_image)
    if results_mesh.multi_face_landmarks:
        mesh_points=np.array([np.multiply([p.x, p.y], [W, H]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
        return mesh_points
    return None

In [5]:
def resize_if_too_small(photo):
    if photo.shape[0] < 300 and photo.shape[1] < 300:
        photo = cv2.resize(photo, (300, 300), interpolation = cv2.INTER_CUBIC)
    return photo

In [6]:
meshes_surprised = []
for image in images_surprised:
    img = cv2.imread('../face_surprised/'+image)
    if img is None:
        continue
    img = resize_if_too_small(img)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_surprised.append(mesh)
print("meshes surprised: ", len(meshes_surprised))

c:\Users\felip\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


meshes surprised:  7373


In [7]:
meshes_disgusted = []
for image in images_disgusted:
    img = cv2.imread('../face_disgusted/'+image)
    if img is None:
        continue
    img = resize_if_too_small(img)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_disgusted.append(mesh)
print("meshes disgusted: ", len(meshes_disgusted))

meshes disgusted:  5421


In [8]:
meshes_sad = []
for image in images_sad:
    img = cv2.imread('../face_sad/'+image)
    if img is None:
        continue
    img = resize_if_too_small(img)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_sad.append(mesh)
print("meshes sad: ", len(meshes_sad))

meshes sad:  7367


In [9]:
meshes_happy = []
for image in images_happy:
    img = cv2.imread('../face_happy/'+image)
    if img is None:
        continue
    img = resize_if_too_small(img)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_happy.append(mesh)
print("meshes happy: ", len(meshes_happy))

meshes happy:  8048


In [10]:
meshes_neutral = []
for image in images_neutral:
    img = cv2.imread('../face_neutral/'+image)
    if img is None:
        continue
    img = resize_if_too_small(img)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_neutral.append(mesh)
print("meshes neutral: ", len(meshes_neutral))

meshes neutral:  8950


In [11]:
meshes_angry = []
for image in images_angry:
    img = cv2.imread('../face_angry/'+image)
    if img is None:
        continue
    img = resize_if_too_small(img)
    mesh = inference_facemesh(img)
    if mesh is not None:
        meshes_angry.append(mesh)
print("meshes angry: ", len(meshes_angry))

meshes angry:  6903


In [12]:
def normalize_landmarks(mesh):
    nose_tip = mesh[4] 
    forehead = mesh[151]
    mesh_norm = mesh - nose_tip
    scale_factor = np.linalg.norm(forehead - nose_tip)
    if np.isclose(scale_factor, 0):
        scale_factor = 1e-6
    mesh_norm = np.divide(mesh_norm, scale_factor)
    landmarks_flat = mesh_norm.flatten()
    return landmarks_flat

In [13]:
meshes_sad_norm = []
for mesh in meshes_sad:
    mesh_norm = normalize_landmarks(mesh)
    meshes_sad_norm.append(mesh_norm)

meshes_happy_norm = []
for mesh in meshes_happy:
    mesh_norm = normalize_landmarks(mesh)
    meshes_happy_norm.append(mesh_norm)

meshes_neutral_norm = []
for mesh in meshes_neutral:
    mesh_norm = normalize_landmarks(mesh)
    meshes_neutral_norm.append(mesh_norm)

meshes_angry_norm = []
for mesh in meshes_angry:
    mesh_norm = normalize_landmarks(mesh)
    meshes_angry_norm.append(mesh_norm)

meshes_surprised_norm = []
for mesh in meshes_surprised:
    mesh_norm = normalize_landmarks(mesh)
    meshes_surprised_norm.append(mesh_norm)

meshes_disgusted_norm = []
for mesh in meshes_disgusted:
    mesh_norm = normalize_landmarks(mesh)
    meshes_disgusted_norm.append(mesh_norm)

In [14]:
df_sad = pd.DataFrame(meshes_sad_norm)
df_happy = pd.DataFrame(meshes_happy_norm)
df_neutral = pd.DataFrame(meshes_neutral_norm)
df_angry = pd.DataFrame(meshes_angry_norm)
df_surprise = pd.DataFrame(meshes_surprised_norm)
df_disgust = pd.DataFrame(meshes_disgusted_norm)

In [15]:
df_sad.to_csv('df_sad.csv', index=False)
df_happy.to_csv('df_happy.csv', index=False)
df_neutral.to_csv('df_neutral.csv', index=False)
df_angry.to_csv('df_angry.csv', index=False)
df_surprise.to_csv('df_surprise.csv', index=False)
df_disgust.to_csv('df_disgust.csv', index=False)